In [9]:
#import packages
import warnings
warnings.simplefilter("ignore")
from keras import backend as K
K.set_image_dim_ordering('tf') # note that we need to have tensorflow dimension ordering still because of the weigths.
print('The backend is:',K.backend())
import tensorflow as tf
print(K.image_dim_ordering()) # should say tf
print(tf.__version__) # tested for 1.11.0
import cv2
import keras
print(keras.__version__) # tested for 2.2.4

The backend is: tensorflow
tf
1.12.0
2.2.4


In [25]:
# Import relevant packages
from __future__ import absolute_import, division, print_function # make it compatible w Python 2
import os
import h5py # to handle weights
import os, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

from PIL import Image

from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation, ZeroPadding2D
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import model_from_json
from keras.preprocessing import image

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

from IPython.display import Image, display

# fix random seed for reproducibility
np.random.seed(150)

In [42]:
data = pd.read_csv("/Users/xudanluo/Desktop/jupyter notebook/290/AI FOR GOOD/streetmap-master/resources.dat")
data

,# URLs pointing our resources
0,https://data.cityofnewyork.us/Public-Safety/NY...


In [26]:
# import maps, note all map images are unique
imgs=[]
for path, dirs, files in os.walk('/Users/xudanluo/Desktop/未命名文件夹'):#file path ending with folder
    #print('FOLDER',path)
    #print("\n",files[1:-1])
    for f in files[1:]:
        imgs.append(cv2.imread(path+"/"+f))
        print(f)
        #imgspath.append(path+"/"+f)
print("samples=",len(imgs))
imgs[-1].shape


testimage_big.png
4.png
2.png
3.png
1.png
xxxx.png
x.png
xxx.png
xx.png
samples= 9


(640, 640, 3)

In [29]:
#inpput labels
labels=[0,1,5,3,4,5,2,1,2]

In [36]:
#split the data
def cross_validate(Xs, ys):
    X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.2, random_state=0)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = cross_validate(imgs, labels)

# confirm we got our data
print(y_test[0:10])


[1, 5]


In [38]:
# normalize inputs from 0-255 and 0.0-1.0
X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# one hot encode outputs
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
#num_classes = y_test.shape[1]

In [39]:
y_train

array([1, 4, 2, 2, 3, 0, 5])

In [ ]:
##潘潘你可以直接改这个或者直接run这个model
input_size=(640,640,3)
num_classes=6

def createCNNModel(num_classes):
#    """ Adapted from: # http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/
# """
    # Create the model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=input_size, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    epochs = 3  # >>> should be 25+
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    return model, epochs

# create our CNN model
model, epochs = createCNNModel(num_classes)
print("CNN Model created.")


In [39]:
#这个模块我今天run，你不用run他
input_size=(640,640,3)
num_classes=6
def createCNNModel(num_classes):
#    """ Adapted from: # http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/
# """
    # Create the model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=input_size, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    epochs = 3  # >>> should be 25+
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    return model, epochs

# create our CNN model
model, epochs = createCNNModel(num_classes)
print("CNN Model created.")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 640, 640, 32)      896       
_________________________________________________________________
dropout_3 (Dropout)          (None, 640, 640, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 320, 320, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3276800)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 19660806  
Total params: 19,661,702
Trainable params: 19,661,702
Non-trainable params: 0
_________________________________________________________________
None
CNN Model created.


In [40]:
X_train.shape

(7, 640, 640, 3)

In [41]:
#batch size，learning rate之类的这些 你跑之前都可以改
batch_size=60
seed = 7
np.random.seed(seed)
model.fit(X_train, y_train, batch_size = batch_size, nb_epoch = epochs)
#model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=epochs, batch_size=60)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

print("done")

Epoch 1/3
7/7 [==============================] - 11s 2s/step - loss: 1.9487 - acc: 0.1429
Epoch 2/3
7/7 [==============================] - 7s 1s/step - loss: 11.5129 - acc: 0.2857
Epoch 3/3
7/7 [==============================] - 7s 1s/step - loss: 11.5129 - acc: 0.2857
Accuracy: 0.00%
done


In [54]:
y_pred=model.predict(X_test)
print("y_pred is ",y_pred)

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [47]:
y_test

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [61]:
##store the weights
for layer in model.layers:
    weights = layer.get_weights() 

In [62]:
weights

[array([[ 1.6228707e-03, -3.4538349e-03,  3.2287699e-03,  1.5444639e-03,
          3.3674845e-03, -5.2891597e-03],
        [-7.8277901e-04, -6.5064844e-04,  6.8643334e-04, -7.4463169e-04,
          6.5404869e-04, -1.8925820e-03],
        [-4.0262926e-04,  1.1396382e-03, -4.4183107e-06,  8.5334317e-04,
         -7.6675159e-04,  9.5519121e-04],
        ...,
        [-1.0649031e-03,  7.1342685e-04,  7.4985437e-04,  1.2977265e-03,
          6.8004243e-04, -7.2314538e-04],
        [ 1.0233629e-03, -4.4777343e-04,  5.0101511e-04,  3.5858125e-04,
         -9.0478320e-04, -1.4484559e-04],
        [ 8.5067679e-04, -6.8056706e-04,  1.0558066e-03,  1.2486810e-03,
          2.8237549e-04,  1.2023586e-03]], dtype=float32),
 array([ 0.00166985, -0.00310872,  0.00293007,  0.00101551,  0.00228385,
        -0.00479055], dtype=float32)]

In [63]:
len(weights)

2

In [64]:
##store the model
model.save('/Users/xudanluo/Desktop/未命名文件夹/my_model.h5')#change path 

In [ ]:
###read the stored the model
import h5py

def print_keras_wegiths(weight_file_path):
    f = h5py.File(weight_file_path)  # 读取weights h5文件返回File类
    try:
        if len(f.attrs.items()):
            print("{} contains: ".format(weight_file_path))
            print("Root attributes:")
        for key, value in f.attrs.items():
            print("  {}: {}".format(key, value))  # 输出储存在File类中的attrs信息，一般是各层的名称

        for layer, g in f.items():  # 读取各层的名称以及包含层信息的Group类
            print("  {}".format(layer))
            print("    Attributes:")
            for key, value in g.attrs.items(): # 输出储存在Group类中的attrs信息，一般是各层的weights和bias及他们的名称
                print("      {}: {}".format(key, value))  

            print("    Dataset:")
            for name, d in g.items(): # 读取各层储存具体信息的Dataset类
                print("      {}: {}".format(name, d.value.shape)) # 输出储存在Dataset中的层名称和权重，也可以打印dataset的attrs，但是keras中是空的
                print("      {}: {}".format(name. d.value))
    finally:
        f.close()